# Introduction to PUMS Notebook

The U.S. Census website  contains numerous kinds of demographic  data about the United States.  In this section we focus on data collected in surveys conducted during the decennial  census, which is distinct from the more frequently collected American Community Survey data.  More especifically, we will  confine our attention to the [Census 2000 PUMS data](http://www2.census.gov/census_2000/datasets/PUMS/>). PUMS  stands for **Public Use Microdata Samples**.  It is called *Microdata* because the the records contain information about one person or one household.

The discussion below assumes you have visited the PUMS ages on the Census Bureau site and downloaded at least two files, `revisedpums1_alabama.txt` and the data dictionary available in the Excel workbooks, `5%_PUMS_record_layout.xls`.  For more details see the book draft chapter on data, especially the section on PUMS data.  You also need the Python module `read_in_census_data`, which can also be downloaded from a link in the PUMS data section of the book draft.

## Reading in the data dictionary

First you need to import the `read_in_census_data` module and read in the data dictionary in
the Census web site Excel sheet.

In [1]:
from read_in_census_data import CensusInfo, DataFrameWrapper


one_percent_data_file = 'revisedpums1_alabama_01.txt'
data_dictionary = '5%_PUMS_record_layout.xls'
ci = CensusInfo (data_dictionary)
#print(ci)

Processing 'Housing Unit Record' Sheet

Processing 'Person Record' Sheet



You can now access information about various variable in the Census data.  You need to supply the variable name (or a legal synonym), the record type (person or household) and the file type (15 or 5%).   Below we look at information for the 'RELATE'
variable, which defines the relationship of a parson to the head of the household.  This is a person record variable (`ci.record_types[1]`) in a 1% file (`ci.file_types[1]`).

In [2]:
ci.print_var_info('RELATE',ci.record_types[1],ci.file_types[1])
ci.print_var_info('RELATE',ci.record_types[1],ci.file_types[0])

Person Record RELATE 1% file {'RT': 'P', 'BEG': 17, 'LEN': 2, 'DESCRIPTION': 'Relationship'}

     Relationship
     01  Householder
     02  Husband/wife
     03  Natural born son/daughter
     04  Adopted son/daughter
     05  Stepson/Stepdaughter
     06  Brother/sister
     07  Father/mother
     08  Grandchild
     09  Parent-in-law
     10  Son-in-law/daughter-in-law
     11  Other relative
     12  Brother-in-law/sister-in-law
     13  Nephew/niece
     14  Grandparent
     15  Uncle/aunt
     16  Cousin
     17  Roomer/boarder
     18  Housemate/roommate
     19  Unmarried partner
     20  Foster child
     21  Other nonrelative
     22  Institutionalized GQ person
     23  Noninstitutionalized GQ person

Person Record RELATE 5% file {'RT': 'P', 'BEG': 17, 'LEN': 2, 'DESCRIPTION': 'Relationship'}

     Relationship
     01  Householder
     02  Husband/wife
     03  Natural born son/daughter
     04  Adopted son/daughter
     05  Stepson/Stepdaughter
     06  Brother/sister
   

In the next code snippet, we get a sample line from the alabama file:

In [3]:
line = ci.get_sample_record_from_db (one_percent_data_file)

Opening revisedpums1_alabama_01.txt
P00001170100014001000010310010110000010147010100100012099999904200000 000101000000300010     01400  70    9997    9997202020202020220000020011000000020031000010     01400  70    9997    9997010100370010000000108190622     313029-1111201052045004730000000000-01500000000000000000000000000000000000004580000047300501


The next code snippets print info about various variables:

In [4]:
#To get info about a DB variable, 3 things must be specfied
# file type (1 per cent or 5 per cent), record type (person or household)
# and the variable name (column name)
(var0,record_type, file_type) = ('RELATE',ci.record_types[1],ci.file_types[1])
# To get avilable info about the VALUE of a variable (usually an opaque code)
# Get the val of this variable for this line
val = ci.get_db_val(line, var0, record_type)
var_values_info = ci.variable_values_dictionary[record_type][(var0,file_type)]
    
# Print some of the info just retrieved.
print (var0,  record_type, val, var_values_info[val])

#ci.print_var_info ('EDUC', ci.person_record)
ci.print_var_info ('education', ci.person_record)
#ci.print_var_info ('RACE3', ci.person_record)
ci.print_var_info ('race', ci.person_record)
ci.print_var_info ('income', ci.person_record)

# Common to both household and person records, links them
ci.print_var_info('serial_no',ci.household_record)

ci.print_var_info('serial_no',ci.person_record)
ci.print_var_info('num_persons',ci.household_record)

try:
  ci.print_var_info('num_persons',ci.person_record)
except Exception as e:
  print(e.message)

RELATE Person Record 01 Householder
Person Record education => EDUC 5% file {'RT': 'P', 'BEG': 53, 'LEN': 2, 'DESCRIPTION': 'Educational Attainment'}

     Educational Attainment
     00  Not in universe (Under 3 years)
     01  No schooling completed
     02  Nursery school to 4th grade
     03  5th grade or 6th grade
     04  7th grade or 8th grade
     05  9th grade
     06  10th grade
     07  11th grade
     08  12th grade, no diploma
     09  High school graduate
     10  Some college, but less than 1 year
     11  One or more years of college, no degree
     12  Associate degree
     13  Bachelor's degree
     14  Master's degree
     15  Professional degree
     16  Doctorate degree

Person Record race => RACE3 5% file {'RT': 'P', 'BEG': 41, 'LEN': 2, 'DESCRIPTION': 'Race Recode 3'}

     Race Recode 3
     01  Some other race alone
     02  Other Pacific Islander alone
     03  Other Pacific Islander; Some other race
     04  Samoan alone
     05  Guamanian or Chamorro alone
 

AttributeError: 'AssertionError' object has no attribute 'message'

## Use the following code to pick out some variables of interest.

In [5]:
for rt in ci.record_types:
    print(rt)
    print('=' * len(rt))
    print()
    rt_dict = ci.data_dictionary[rt]
    banner = '%-5s %14s  %s' % ('Var', 'Synonym', 'Description')
    print(banner)
    print('=' * len(banner))
    print()
    for var in sorted(rt_dict.keys()):
        syns = ', '.join(ci.backward_syns[var])
        print('%-9s %10s  %s' % (var, syns, rt_dict[var]['DESCRIPTION']))
    print()

Housing Unit Record

Var          Synonym  Description

ACRES                 Acreage
ACRESA                Acreage Allocation Flag
AGSALES               Sales of Agricultural Products in 1999
AGSALESA              Sales of Agricultural Products in 1999 Allocation Flag
AREATYP1              Metropolitan Area: SuperPUMA Relationship to MA
AREATYP5              Metropolitan Area: PUMA Relationship to MA
BEDRMS                Number of Bedrooms
BEDRMSA               Number of Bedrooms Allocation Flag
BLDGSZ                Size of Building
BLDGSZA               Size of Building Allocation Flag
BUSINES               Commercial Business on Property
BUSINESA              Commercial Business on Property Allocation Flag
CKITCH                Complete Kitchen Facilities
CKITCHA               Complete Kitchen Facilities Allocation Flag
CONDFEE               Condominium Fee (monthly)
CONDFEEA              Condominium Fee (monthly) Allocation Flag
CPLUMB                Complete Plumbing Facilities


## Learn more about the values of variable of interest with the following code.

But remember you have to get the record type right.  The following fails because "INCWS" (Wage/salaray income) is not
a variable for HOUSHOLD records.

In [6]:
ci.print_var_info('INCWS',ci.household_record)

AssertionError: INCWS is not a known variable for a Housing Unit Record

In [7]:
ci.print_var_info('INCWS',ci.person_record)

Person Record INCWS 5% file {'RT': 'P', 'BEG': 244, 'LEN': 6, 'DESCRIPTION': 'Wage/Salary Income in 1999'}

     Wage/Salary Income in 1999
     ('000000', ' ')  No/none
     ('000001', '174999')  $1 to $174,999
     175000  Topcode
     175000+  State mean of topcoded values
      blank  Not in universe (Under 15 years)



Values at or above topcode are not directly represented.  Instead the mean for values above topcode (for that particular state) are substituted.  You can find out what these means are for topcoded housing and person records [here](http://webapp1.dlib.indiana.edu/virtual_disk_library/index.cgi/5688252/FID3502/Document/Appendix%20H.txt).  In addition, each variable has a **universe**, a set of entities for which it is supposed to represent information.  For the Wage salary income variable INCWS, that universe is persons 15 years of age or over.  Persons under 15 years of age have a blank in that column.  That represents different information than a 0 (which means a person 15 or older earning no salary income).  What is the right thing to do with such records  when computing average salaries?

Let's do gender next.

In [8]:
ci.print_var_info('gender',ci.person_record)

Person Record gender => SEX 5% file {'RT': 'P', 'BEG': 23, 'LEN': 1, 'DESCRIPTION': 'Sex'}

     Sex
     ===
     1  Male
     2  Female



Many variable are just BOOLEAN variables.  They represent whether or not the person or household falls in some category with a 1 or a 2.  The MENTAL (mental disability) variable shown next is an example.  Variables are sometimes accompanied by **allocation** variables, which give information about how the value of that variable for that particular record was arrived at.  For example the MENTAL variable has an accompanying MENTALA allocation variable, which can have the value 1 or 0, with a 1 indicating that the value of the MENTAL variable was missing in the raw data and was inferred by one of a number of allocation methods outlined [here](https://usa.ipums.org/usa/flags.shtml).  Whether or not allocated data should be excluded is a complicated subject having to do with the methods used for allocation and whether a bias may have been introduced.

In [9]:
ci.print_var_info('MENTAL',ci.person_record)

Person Record MENTAL 5% file {'RT': 'P', 'BEG': 123, 'LEN': 1, 'DESCRIPTION': 'Mental Disability'}

     Mental Disability
     1  Yes
     2  No 
     blank  Not in universe (Under 5 years)



In [ ]:
ci.print_var_info('MENTALA',ci.person_record)

In [ ]:
ci.print_var_info('PHYSCL',ci.person_record)

In [ ]:
ci.print_var_info('PHYSCLA',ci.person_record)

More verbose data is printed out for a variable that has a complex value set.

In [ ]:
ci.print_var_info('education',ci.person_record)

In some cases information about a code value must be gotten by going back to [the Census 2000 PUMS data](http://www2.census.gov/census_2000/datasets/PUMS/)  and downloading some further documentation.  This is true of two very important types of information, occupation and location.  For example, information about the metropolitan area of a household is coded using FIPS MSA/CMSA codes.  The coding explanations are stored in documents available state by state and named according to a standard convention. The name template is "PUMEQ[filetype]-[StateCode].TXT", where filetype is 1 (for 1%) or 5 (for 5%), and "StateCode" is a standard two-letter state code.

So, for example, the file giving all the location code info for the 1% file for Alabmama is named "PUMEQ1-AL.TXT".  So the geographical meaning of code '0040-9360', the MSA/CMSA codes shown below, will be found there.  

In [ ]:
ci.print_var_info('MSACMSA1',ci.household_record)

On the other hand, although the record below suggests we look in something called Appendix G for the language codes, they actually appear to be stored in a file called 5%_PUMS_language.xls.  The codes are the same for 1% and 5% data, and so a lot of this auxiliaryu documentation is stored only in the 5% portion of the website.

In [ ]:
ci.print_var_info('LANG1',ci.person_record)

## Selecting a subset of the data

In [10]:
df = DataFrameWrapper(ci)
# we select a subset of the columns.
df.fill_frame ([], ci.person_record, one_percent_data_file, ci.one_percent_file, \
                'race','education','income','gender','age','relationship')
df.rows[:5]

[['0000117', '47', '12', '047300', '1', '31', '01'],
 ['0000117', '47', '12', '024300', '2', '25', '02'],
 ['0000117', '47', '00', '      ', '1', '00', '03'],
 ['0000127', '47', '05', '000000', '1', '22', '01'],
 ['0000127', '47', '10', '022000', '2', '35', '02']]

Note that the household serial_no field is included automatically, because this is the column
that let us link this to other records (to other persons in the same household, so that
we can aggregate information about households.

In [11]:
df.header

['serial_no', 'race', 'education', 'income', 'gender', 'age', 'relationship']

Order of the original rows is preserved so the first two person records are about two people in the
same household.

In [12]:
df.rows[:5]

[['0000117', '47', '12', '047300', '1', '31', '01'],
 ['0000117', '47', '12', '024300', '2', '25', '02'],
 ['0000117', '47', '00', '      ', '1', '00', '03'],
 ['0000127', '47', '05', '000000', '1', '22', '01'],
 ['0000127', '47', '10', '022000', '2', '35', '02']]

In [13]:
df.save_frame('alabama_pums_extract.csv',header=True)

In [14]:
pwd

'/Users/gawron/ext/src/sphinx/python_for_ss/ipython_notebooks/pandas'

In [16]:
import pandas as pd
import numpy as np
## Note default is to use header line to name cols.
## header argument is for selecting cols or renaming cols
## Specify dat types for all the codes that arent really intended to be treated as numbers
## Let pandas try to figure out income,education, and age.
dt_dict ={'serial_no':object,'race':object,'gender':object,'relationship':object}
p_df = pd.read_csv('alabama_pums_extract.csv',
                   dtype=dt_dict)
#p_df[:5]

Just for practice, save this file.  The version we're saving is identical
to the one we just read in.

In [17]:
p_df.to_csv('alabama_pums_extract3.csv',index=False)

In [18]:
p_df

,serial_no,race,education,income,gender,age,relationship
0,0000117,47,12,47300.0,1,31,01
1,0000117,47,12,24300.0,2,25,02
2,0000117,47,0,NaN,1,0,03
3,0000127,47,5,0.0,1,22,01
4,0000127,47,10,22000.0,2,35,02
5,0000127,47,5,0.0,1,19,05
6,0000127,47,7,5000.0,1,18,17
7,0000134,47,13,13000.0,2,23,01
8,0000195,47,10,0.0,1,56,01
9,0000195,47,9,0.0,2,56,02


Let's look at a sample record.  Note the use of `iloc`.  Along with `loc`, this is one of
Panda's two most important indexing methods.  It allows both numerical and name based indexing.
Here we just use a number to look at the 279th row:

In [19]:
p_df.iloc[278]

serial_no       0006874
race                 47
education            11
income            18600
gender                2
age                  31
relationship         01
Name: 278, dtype: object

So this is a 31 year old white male head of household (race code 47 =   white alone, gender 1 = male,
relationship 1 = head of household, with an income of $18,600, who completed
one or more years of college but did not receive a degree.

## Appendix: Data types

In [20]:
p_df.dtypes

serial_no        object
race             object
education         int64
income          float64
gender           object
age               int64
relationship     object
dtype: object

Notice income,education, and age are integers and floats respectively,  That's good because we're going to want to do aggregation operations like mean and sum on income.  We're going to use the numerical values in education to bin education levels.
Below we single out the class of folks with high school diplomas or above (`EDUCATION` >= 9).

Let's look at how to change data types, which we'll need to do below.  First we cook up a
simple example.

In [38]:
df2 = pd.DataFrame([['1','2'],['34','56']],columns= ['h','t'])
              

df2.dtypes

h    object
t    object
dtype: object

In [39]:
print(type(df2['h'][0]))
df2['h'][0]

<class 'str'>


'1'

In [40]:
df2['h'].astype(int)

0     1
1    34
Name: h, dtype: int64

In [41]:
df2['h'] = df2['h'].astype(int)

In [42]:
print(type(df2['h'][0]))
df2['h'][0]

<class 'numpy.int64'>


1

## Aggregation

In [38]:
p_df

,serial_no,race,education,income,gender,age,relationship
0,0000117,47,12,47300.0,1,31,01
1,0000117,47,12,24300.0,2,25,02
2,0000117,47,0,NaN,1,0,03
3,0000127,47,5,0.0,1,22,01
4,0000127,47,10,22000.0,2,35,02
5,0000127,47,5,0.0,1,19,05
6,0000127,47,7,5000.0,1,18,17
7,0000134,47,13,13000.0,2,23,01
8,0000195,47,10,0.0,1,56,01
9,0000195,47,9,0.0,2,56,02


In [39]:
p_df.columns

Index(['serial_no', 'race', 'education', 'income', 'gender', 'age',
       'relationship'],
      dtype='object')

In [40]:
p_df[p_df['race'] == '47']

,serial_no,race,education,income,gender,age,relationship
0,0000117,47,12,47300.0,1,31,01
1,0000117,47,12,24300.0,2,25,02
2,0000117,47,0,NaN,1,0,03
3,0000127,47,5,0.0,1,22,01
4,0000127,47,10,22000.0,2,35,02
5,0000127,47,5,0.0,1,19,05
6,0000127,47,7,5000.0,1,18,17
7,0000134,47,13,13000.0,2,23,01
8,0000195,47,10,0.0,1,56,01
9,0000195,47,9,0.0,2,56,02


We map the values in the `race` column to Booleans using a Boolean test.  We will use resulting Pandas `Series` to create a white/non-white column.

In [41]:
p_df['race'].map(lambda x: x == '47')

0         True
1         True
2         True
3         True
4         True
5         True
6         True
7         True
8         True
9         True
10        True
11       False
12        True
13        True
14        True
15        True
16        True
17        True
18       False
19       False
20       False
21        True
22        True
23        True
24        True
25        True
26       False
27       False
28       False
29       False
         ...  
44457    False
44458     True
44459    False
44460    False
44461    False
44462    False
44463     True
44464     True
44465     True
44466     True
44467     True
44468     True
44469     True
44470     True
44471     True
44472     True
44473     True
44474     True
44475     True
44476    False
44477    False
44478    False
44479     True
44480     True
44481    False
44482    False
44483    False
44484    False
44485    False
44486    False
Name: race, Length: 44487, dtype: bool

In [44]:
p_df['white_only'] = p_df['race'].map(lambda x: x == '47')

In [43]:
p_df

,serial_no,race,education,income,gender,age,relationship,white_only
0,0000117,47,12,47300.0,1,31,01,True
1,0000117,47,12,24300.0,2,25,02,True
2,0000117,47,0,NaN,1,0,03,True
3,0000127,47,5,0.0,1,22,01,True
4,0000127,47,10,22000.0,2,35,02,True
5,0000127,47,5,0.0,1,19,05,True
6,0000127,47,7,5000.0,1,18,17,True
7,0000134,47,13,13000.0,2,23,01,True
8,0000195,47,10,0.0,1,56,01,True
9,0000195,47,9,0.0,2,56,02,True


We can now do grouping operations to create a `groupby` series that separates white from non-white households.

In [45]:
grouped0 = p_df['income'].groupby(p_df['white_only'])

In [46]:
grouped0

In [45]:
grouped0.mean()

white_only
False    11294.922766
True     17556.744213
Name: income, dtype: float64

This number seems rather low.  One reason is we're including those people outside the universe
in the calculation (5 year old children, for example), giving them income 0.
We fix this as follows:

In [47]:
grouped2 = p_df[p_df['income']>0]['income'].groupby(p_df['white_only'])

Let's break this apart into pieces.

In [47]:
p_df[p_df['income']>0]

,serial_no,race,education,income,gender,age,relationship,white_only
0,0000117,47,12,47300.0,1,31,01,True
1,0000117,47,12,24300.0,2,25,02,True
4,0000127,47,10,22000.0,2,35,02,True
6,0000127,47,7,5000.0,1,18,17,True
7,0000134,47,13,13000.0,2,23,01,True
10,0000218,47,9,21800.0,1,20,01,True
11,0000246,37,14,32000.0,2,26,01,False
12,0000410,47,13,56000.0,1,34,01,True
13,0000436,47,13,42000.0,1,46,01,True
14,0000436,47,14,70000.0,2,45,02,True


This gives the same 8-column table with records that have no income filtered out, so it's a much shorter table.

In [48]:
p_df[p_df['income']>0]['income']

0         47300.0
1         24300.0
4         22000.0
6          5000.0
7         13000.0
10        21800.0
11        32000.0
12        56000.0
13        42000.0
14        70000.0
18        24000.0
21        17000.0
22         7000.0
28          750.0
30       148000.0
32         1400.0
35        42000.0
36        26700.0
40        31200.0
41        12400.0
44        24000.0
45        17000.0
46         1200.0
47        30000.0
48         6000.0
49        24000.0
50        23000.0
51         1800.0
54        14000.0
56         9800.0
           ...   
44419     28500.0
44420     65000.0
44421     21900.0
44422      3800.0
44423      8000.0
44434     50000.0
44435     27900.0
44439     15000.0
44441     23000.0
44442      3000.0
44445     23500.0
44447     21300.0
44448      1400.0
44451     24000.0
44453     41000.0
44454      7500.0
44455     29000.0
44456      6000.0
44457     14000.0
44458     45000.0
44459     33000.0
44466     30000.0
44467     15900.0
44469     12000.0
44470     

Now we've got just the income column of that table.  Now we do a `split` step.  For one group 'white_only' = True and for the other,'white_only = False'.

In [48]:
grouped2 = p_df[p_df['income']>0]['income'].groupby(p_df['white_only'])

The grouping operation is separated from the step where we apply some operation to the groups to get numbers.  For example, let's take mean.

In [50]:
grouped2.mean()

white_only
False    20276.427615
True     29245.365239
Name: income, dtype: float64

And this time let's sum the incomes in the groups.

In [51]:
grouped2 = p_df.groupby(['white_only'])

In [52]:
totals2 = grouped2.income.sum().fillna(0)

In [53]:
totals2

white_only
False    106025440.0
True     452805990.0
Name: income, dtype: float64

Since we have only numerical columns we can also do means for all the columns:

In [54]:
totals3 = grouped2.mean().fillna(0)

In [55]:
totals3

,education,income,age
white_only,,,
False,6.739302,11294.922766,32.321223
True,7.884836,17556.744213,38.566922


A groupby object has keys that correspond to the values in the original column we grouped by.

In [57]:
list(totals3.keys())

['education', 'income', 'age']

In [58]:
grouped2 = p_df[p_df['income']>0]['income'].groupby(p_df['gender'])

In [59]:
grouped2.mean().fillna(0)

gender
1    33706.836926
2    19834.642501
Name: income, dtype: float64

## A more complex example (education)

In [60]:
p_df['white_only'] = p_df['race'].map(lambda x: x == '47')
p_df['high_school'] = p_df['education'].map(lambda x: x >= 9)
p_df2 = p_df.loc[:,['high_school','white_only']]

We make a data frame consisting of just the columns we're interested in using `loc`.

In [61]:
p_df2

,high_school,white_only
0,True,True
1,True,True
2,False,True
3,False,True
4,True,True
5,False,True
6,False,True
7,True,True
8,True,True
9,True,True


We now want counts for the entire data set of each pairing of the high_school diploma
variable with the white_only variable.  This is done with `crosstab`.

In [63]:
xtab = pd.crosstab(p_df2.white_only,p_df2.high_school,margins=True)
xtab

high_school,False,True,All
white_only,,,
False,7059,5630,12689
True,12956,18842,31798
All,20015,24472,44487


Turning these into percentages turns out to be a little harder than I thought.  Maybe there's
an easier way?  Intuitively, we want to divide each column by the last column, so that all the entries
in the False row are divided by 12689 and all the entries in the True row by 31798 and all the entries
in the All row by 44487.  First we need a version of the last column that is floats rather than integers:

In [64]:
all_f = xtab['All'].map(lambda x: float(x))
all_f

white_only
False    12689.0
True     31798.0
All      44487.0
Name: All, dtype: float64

Next we'll just divide each column of `xtab` in turn by `all_f`, resetting each column to
be the resulting column of percentages:

In [65]:
xtab[False] = xtab[False]/all_f
xtab[True] = xtab[True]/all_f
xtab['All'] = xtab['All']/all_f
xtab
#xtab.columns

high_school,False,True,All
white_only,,,
False,0.556309,0.443691,1.0
True,0.407447,0.592553,1.0
All,0.449907,0.550093,1.0


## Levels of education

The next example is very important.  We make a new column defined via a function that computes something based on the value of  one of the existing columns.  The function is called `education`. It sorts education levels into various bins we will use to define a new column called `degrees`.

In [55]:
def education (x):
    
    if 9 <= x <= 11:
        return 'HS'
    elif x == 12:
        return 'AA'
    elif x == 13:
        return 'BA'
    elif x == 14:
        return 'MA'
    elif x > 14:
        return 'HD'
    else:
        return 'ND'
p_df_inc = p_df[p_df['income']>0]
p_df_inc['degrees'] = p_df_inc['education'].map(education)
p_df_inc2 = p_df_inc.loc[:,['degrees','income']]

/Users/gawron/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [56]:
p_df_inc2

,degrees,income
0,AA,47300.0
1,AA,24300.0
4,HS,22000.0
6,ND,5000.0
7,BA,13000.0
10,HS,21800.0
11,MA,32000.0
12,BA,56000.0
13,BA,42000.0
14,MA,70000.0


In [57]:
p_df_inc2.loc[278]

degrees       HS
income     18600
Name: 278, dtype: object

In [58]:
grouped3 = p_df_inc2.groupby(['degrees'])

In [59]:
grouped3.mean()

,income
degrees,
AA,29264.865269
BA,42062.469136
HD,84993.113456
HS,23209.507226
MA,44152.826718
ND,17061.500257


What degree adds the least value?

## Assignment

### Find the mean income by gender.

### Using section 1.7 ("A more complex example (education)" as a model, construct a table showing the percentages of those receiving a high school education or better for males and females.

In [60]:
gender_group=p_df['income'].groupby(p_df['gender'])
gender_group.mean()

gender
1    21876.639294
2    10629.708614
Name: income, dtype: float64

In [61]:
gender_group=p_df[p_df['income']>0]['income'].groupby(p_df['gender'])
gender_group.mean()

gender
1    33706.836926
2    19834.642501
Name: income, dtype: float64

In [62]:
#p_df['gender']
#Tried to retain the gender values as 1 and 2, it results in 'NaN'
#mapped the gender values to Booleans using a Boolean test by 
#using value of male male
p_df['Male'] = p_df['gender'].map(lambda x: x == '1')
p_df['high_school'] = p_df['education'].map(lambda x: x >= 9)
p_df3 = p_df.loc[:,['high_school','Male']]
p_df3


,high_school,Male
0,True,True
1,True,False
2,False,True
3,False,True
4,True,False
5,False,True
6,False,True
7,True,False
8,True,True
9,True,False


In [69]:
#x1tab = pd.crosstab(p_df3.gender,p_df3.high_school,margins=True)
#The above code 
x1tab = pd.crosstab(p_df3.Male,p_df3.high_school,margins=True)
x1tab

high_school,False,True,All
Male,,,
False,10041,13184,23225
True,9974,11288,21262
All,20015,24472,44487


In [71]:
all_f = x1tab['All']
all_f

Male
False    23225
True     21262
All      44487
Name: All, dtype: int64

In [72]:
x1tab[False] = x1tab[False]/all_f
x1tab[True] = x1tab[True]/all_f
x1tab['All'] = x1tab['All']/all_f
x1tab

high_school,False,True,All
Male,,,
False,0.432336,0.567664,1.0
True,0.469100,0.530900,1.0
All,0.449907,0.550093,1.0


The above table displays the percent of male and female without and with a high school degree respectively. It is evident that the percent of female without high school degree is less at 43.23% when compared to male who stands at 46.91%.


## Potential projects

Here are some ideas for potential projects using the PUMS data.

1) Compare income levels for a variety of levels of education state by state. This can be done
   by creating bins for education level as in the example above.  You may wish to explore different
   bins than the ones used in that example.  For this project you must do at least 10 states, and you must
   choose states that represent some variety in population and urbanization.  You will need to turn in the Python
   notebook, some prose and explaining and justifying what you did (2 pages), and you need to produce some graphs
   plotting education level (x axis) and income levels (y axis).  Make sure you can get the data for
   several states on one plot.  Use different colored lines.  Your intellectual goal is  to decide if differences in
   education level explain the differences in income state by state, or whether there are other major factors. An
   alternative visualization is to use maps like the maps we use for the campaign contribution to represent (state by state)
   percentages of residents with a particular level of education reaching a certain income level. 
   
2) Compare income levels for a variety of levels of education for whites and non-whites.  You can do this in
   several states, optionally but you must do at least one.  You will need to turn in the Python
   notebook, some prose and explaining and justifying what you did (2 pages), and you need to produce some graphs
   plotting education level (x axis) and income levels (y axis).  Make sure you can get the data for
   whites and non-whites on one plot.  Use different colored lines.  Your intellectual goal is  to decide if differences in
   education level explain the differences in income for whites and non whites, or whether there are other major factors.
   
   